In [1]:
import pandas as pd 
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
wiki_data = pd.read_csv('../GitHub_data/people_wiki/people_wiki.csv')
wiki_data.head()

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...


In [3]:
vectorizer = CountVectorizer()
word_counts = vectorizer.fit_transform(wiki_data['text'])

In [5]:
wiki_data.shape

(59071, 3)

In [9]:
vectorizer.get_feature_names()[-20:]

['zysman',
 'zysmans',
 'zyta',
 'zytek',
 'zytomierska',
 'zyttrum',
 'zyuganov',
 'zyuganovs',
 'zyurt',
 'zywo',
 'zyx',
 'zyzzyva',
 'zyzzyza',
 'zz',
 'zzap64',
 'zzb',
 'zzebra',
 'zzran',
 'zzt',
 'zzts']

In [11]:
type(word_counts)

scipy.sparse.csr.csr_matrix

In [12]:
from sklearn.neighbors import KNeighborsClassifier

In [13]:
knn_model = KNeighborsClassifier()
knn_model.fit(word_counts , wiki_data['name'])

KNeighborsClassifier()

In [14]:
# Index of Joe Biden
wiki_data[wiki_data['name'] == 'Joe Biden']

,URI,name,text
24478,<http://dbpedia.org/resource/Joe_Biden>,Joe Biden,joseph robinette joe biden jr dosf rbnt badn b...


In [17]:
jb_sparse = vectorizer.transform(wiki_data[wiki_data['name'] == 'Joe Biden']['text'])

In [23]:
neighbor_dist = knn_model.kneighbors(jb_sparse,n_neighbors=10)[0][0]
neighbor_index = knn_model.kneighbors(jb_sparse,n_neighbors=10)[1][0]

In [24]:
jb_neighbors = pd.DataFrame({'distance' : neighbor_dist , 'index' : neighbor_index})

In [25]:
jb_neighbors

,distance,index
0,0.000000,24478
1,28.740216,8031
2,29.000000,5420
3,29.086079,35721
4,29.223278,46179
5,29.308702,35541
6,29.308702,8002
7,29.410882,47303
8,29.529646,39357
9,29.916551,34214


In [26]:
jb_neighbors = pd.merge(jb_neighbors , wiki_data['name'].reset_index() , on='index' , how='left')

In [27]:
jb_neighbors

,distance,index,name
0,0.000000,24478,Joe Biden
1,28.740216,8031,William R. Hawkins
2,29.000000,5420,David Whissell
3,29.086079,35721,Michael Henderson (author)
4,29.223278,46179,Mike Gravel
5,29.308702,35541,Jimmy Carter
6,29.308702,8002,Giancarlo Pagliarini
7,29.410882,47303,John Kerry
8,29.529646,39357,John McCain
9,29.916551,34214,Matthew Steen


In [28]:
# Now let's compare top 10 neighbors of Joe Biden using TfIdf
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_count = tfidf.fit_transform(wiki_data['text'])

In [32]:
knn_model_tfidf = KNeighborsClassifier()
knn_model_tfidf.fit(tfidf_count , wiki_data['name'])

KNeighborsClassifier()

In [34]:
jb_sparse_tfidf = tfidf.transform(wiki_data[wiki_data['name'] == 'Joe Biden']['text'])

In [35]:
jb_neighbors_tfidf_distance = knn_model_tfidf.kneighbors(jb_sparse_tfidf,n_neighbors=10)[0][0]
jb_neighbors_tfidf_index = knn_model_tfidf.kneighbors(jb_sparse_tfidf,n_neighbors=10)[1][0]

In [36]:
jb_neighbors_tfidf = pd.DataFrame({'distance' : jb_neighbors_tfidf_distance , 'index' : jb_neighbors_tfidf_index})
jb_neighbors_tfidf

,distance,index
0,2.356080e-08,24478
1,1.036251e+00,29596
2,1.101389e+00,16880
3,1.165145e+00,35817
4,1.167491e+00,55880
5,1.212316e+00,23309
6,1.251899e+00,4032
7,1.270501e+00,19152
8,1.282683e+00,46811
9,1.287690e+00,40353


In [37]:
jb_neighbors_tfidf = pd.merge(jb_neighbors_tfidf , wiki_data['name'].reset_index() , on='index', how='left')

In [38]:
jb_neighbors_tfidf

,distance,index,name
0,2.356080e-08,24478,Joe Biden
1,1.036251e+00,29596,Jill Biden
2,1.101389e+00,16880,Cynthia Hogan
3,1.165145e+00,35817,Barack Obama
4,1.167491e+00,55880,Sheila Nix
5,1.212316e+00,23309,Chris Coons
6,1.251899e+00,4032,Kenneth D. Thompson
7,1.270501e+00,19152,Michael Castle
8,1.282683e+00,46811,Jeff Sessions
9,1.287690e+00,40353,Neil MacBride


In [49]:
jb_mat = jb_sparse.toarray()
jb_mat.shape

(1, 548429)

In [50]:
jb_mat.argsort(axis = 1)[:,-10:]

array([[ 45073, 487995, 233774,  89034, 525845, 494541, 360068, 251905,
         58906, 488148]], dtype=int64)

In [59]:
np.array(vectorizer.get_feature_names())[jb_mat.argsort(axis = 1)[:,-10:]]

array([['act', 'that', 'he', 'biden', 'was', 'to', 'of', 'in', 'and',
        'the']], dtype='<U2140')

In [63]:
pd.DataFrame(np.array(vectorizer.get_feature_names())[jb_mat.argsort(axis = 1)[:,-10:]][0][::-1] , columns=['top_10_words'])

,top_10_words
0,the
1,and
2,in
3,of
4,to
5,was
6,biden
7,he
8,that
9,act


In [60]:
jb_mat_tfidf = jb_sparse_tfidf.toarray()
jb_mat_tfidf.shape

(1, 548115)

In [61]:
jb_mat_tfidf.argsort(axis = 1)[:,-10:]

array([[269252, 442385, 155024, 395397, 442467, 417356, 518147,  45070,
        358378,  88985]], dtype=int64)

In [62]:
np.array(tfidf.get_feature_names())[jb_mat_tfidf.argsort(axis = 1)[:,-10:]]

array([['judiciary', 'senate', 'delaware', 'president', 'senator',
        'resolved', 'vice', 'act', 'obama', 'biden']], dtype='<U2140')

In [64]:
pd.DataFrame(np.array(tfidf.get_feature_names())[jb_mat_tfidf.argsort(axis = 1)[:,-10:]][0][::-1] , columns=['top_10_words'])

,top_10_words
0,biden
1,obama
2,act
3,vice
4,resolved
5,senator
6,president
7,delaware
8,senate
9,judiciary
